First I have to use google drive to get the dataset in since it is ~6.5gb which is too much for colab
Source: https://www.freecodecamp.org/news/how-to-transfer-large-files-to-google-colab-and-remote-jupyter-notebooks-26ca252892fa/

In [1]:
!pip install PyDrive

In [2]:
from google.colab import drive

In [3]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
!ls '/content/gdrive/MyDrive'

 111incl2_2020.gdoc			      'outline for formal report.gdoc'
 111incl2_2020.pdf			       UCF101.rar
'Chair concepts.gdoc'			      'Untitled document (1).gdoc'
'Colab Notebooks'			      'Untitled document (2).gdoc'
'Copy of P-11 Poster Layout.gslides'	      'Untitled document.gdoc'
'Experiment 1 Shovel or Claw.gdoc'	      'Untitled spreadsheet.gsheet'
'Getting started.pdf'			      'WDM of Claw.gsheet'
'Google doc'				       WilliamLaurence.docx
'Module 2 Presentation.gslides'		       WilliamReading.docx
'Module 2 Presentation - Pie chart 1.gsheet'


For reference, this next step took me about 8 minutes

In [5]:
!pip install pyunpack
!pip install patool
from pyunpack import Archive
Archive('/content/gdrive/MyDrive/UCF101.rar').extractall('/content')

     |████████████████████████████████| 81kB 7.3MB/s 


Everything after this point is based off this source:
https://www.analyticsvidhya.com/blog/2019/09/step-by-step-deep-learning-tutorial-video-classification-python/

Now i'm setting up the dataset

In [6]:
import cv2     # for capturing videos
import math   # for mathematical operations
import matplotlib.pyplot as plt    # for plotting the images
%matplotlib inline
import pandas as pd
from keras.preprocessing import image   # for preprocessing the images
import numpy as np    # for mathematical operations
from keras.utils import np_utils
from skimage.transform import resize   # for resizing images
from sklearn.model_selection import train_test_split
from glob import glob
from tqdm import tqdm

In [7]:
# open the .txt file which have names of training videos
f = open("/content/trainlist01.txt", "r")
temp = f.read()
videos = temp.split('\n')

# creating a dataframe having video names
train = pd.DataFrame()
train['video_name'] = videos
train = train[:-1]
train.head()

,video_name
0,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c01.avi 1
1,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c02.avi 1
2,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c03.avi 1
3,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c04.avi 1
4,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c05.avi 1


In [8]:
# open the .txt file which have names of test videos
f = open("/content/testlist01.txt", "r")
temp = f.read()
videos = temp.split('\n')

# creating a dataframe having video names
test = pd.DataFrame()
test['video_name'] = videos
test = test[:-1]
test.head()

,video_name
0,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c01.avi
1,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c02.avi
2,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c03.avi
3,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c04.avi
4,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c05.avi


In [9]:
# creating tags for training videos
train_video_tag = []
for i in range(train.shape[0]):
    train_video_tag.append(train['video_name'][i].split('/')[0])
    
train['tag'] = train_video_tag

# creating tags for test videos
test_video_tag = []
for i in range(test.shape[0]):
    test_video_tag.append(test['video_name'][i].split('/')[0])
    
test['tag'] = test_video_tag

In [10]:
# storing the frames from training videos
for i in tqdm(range(train.shape[0])):
    count = 0
    videoFile = train['video_name'][i]
    cap = cv2.VideoCapture('/content/UCF-101/'+videoFile.split(' ')[0])   # capturing the video from the given path

    frameRate = cap.get(5) #frame rate
    x=1

    while(cap.isOpened()):
     
        frameId = cap.get(1) #current frame number
        ret, frame = cap.read()
        if (ret != True):
            break
        if (frameId % math.floor(frameRate) == 0):
            # storing the frames in a new folder named train_1
            filename ='/train_1/' + videoFile.split('/')[1].split(' ')[0] +"_frame%d.jpg" % count;count+=1
            cv2.imwrite(filename, frame)
    cap.release()

100%|██████████| 9537/9537 [10:20<00:00, 15.37it/s]


In [11]:
# getting the names of all the images
images = glob("/train_1/*.jpg")
train_image = []
train_class = []
for i in tqdm(range(len(images))):
   
    # creating the image name
    train_image.append(images[i].split('/')[2])
  
    # creating the class of image
    train_class.append(images[i].split('/')[2].split('_')[1])
    
# storing the images and their class in a dataframe
train_data = pd.DataFrame()
train_data['image'] = train_image
train_data['class'] = train_class

# converting the dataframe into csv file 
train_data.to_csv('/content/train_new.csv',header=True, index=False)

100%|██████████| 73844/73844 [00:00<00:00, 685421.57it/s]


Reading all the video frames

In [12]:
import keras
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, InputLayer, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [13]:
train = pd.read_csv('/content/train_new.csv')
train.head()

,image,class
0,v_SkateBoarding_g15_c06.avi_frame3.jpg,SkateBoarding
1,v_Billiards_g13_c06.avi_frame4.jpg,Billiards
2,v_ApplyLipstick_g10_c04.avi_frame3.jpg,ApplyLipstick
3,v_Skiing_g14_c04.avi_frame6.jpg,Skiing
4,v_WritingOnBoard_g20_c03.avi_frame5.jpg,WritingOnBoard


In [18]:
# creating an empty list
train_image = []

# for loop to read and store frames
for i in tqdm(range(1500)):
  
    # loading the image and keeping the target size as (224,224,3)
    img = image.load_img('/train_1/'+train['image'][i], target_size=(224,224,3))
    # converting it to array
    img = image.img_to_array(img)
    # normalizing the pixel value
    img = img/255
    # appending the image to the train_image list
    train_image.append(img)
    
# converting the list to numpy array
X = np.array(train_image)

# shape of the array
X.shape

100%|██████████| 1500/1500 [00:02<00:00, 502.22it/s]


(1500, 224, 224, 3)

Was supposed to have a sample size of 73844, but my ram couldn't hold even 30 % of that and I still need a lot of it for future operations so I'll only do 900 samples. We're only testing whether we can actually put a model onto a website that inputs videos anyway so it shouldn't matter how correct our model is right now.

Expected Output: (73844, 224, 224, 3)

Creating a validation set

In [19]:
# separating the target
y = train['class']
y = y[:1500]


# creating the training and validation set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2, stratify = y)

In [20]:
# creating dummies of target variable for train and validation set
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

Defining the architecture of the model

In [21]:
# creating the base model of pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False)

58892288/58889256 [==============================] - 1s 0us/step


In [22]:
# extracting features for training frames
X_train = base_model.predict(X_train)
X_train.shape

(1200, 7, 7, 512)

Output is different since sample size was reduced

Expected Output: (59075, 7, 7, 512)

In [23]:
# extracting features for validation frames
X_test = base_model.predict(X_test)
X_test.shape

(300, 7, 7, 512)

Output is different since sample size was reduced

Expected Output: (14769, 7, 7, 512)

In [24]:
# reshaping the training as well as validation frames in single dimension
X_train = X_train.reshape(1200, 7*7*512)
X_test = X_test.reshape(300, 7*7*512)

In [25]:
# normalizing the pixel values
max = X_train.max()
X_train = X_train/max
X_test = X_test/max

In [26]:
# shape of images
X_train.shape

(1200, 25088)

Expected Output: (59075, 25088)

In [27]:
#defining the model architecture
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(25088,)))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(101, activation='softmax'))

Training the video classification model

In [28]:
# defining a function to save the weights of best model
from keras.callbacks import ModelCheckpoint
mcp_save = ModelCheckpoint('weight.hdf5', save_best_only=True, monitor='val_loss', mode='min')

In [29]:
# compiling the model
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [32]:
# training the model
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), callbacks=[mcp_save], batch_size=128)

Epoch 1/10
10/10 [==============================] - 0s 28ms/step - loss: 4.3861 - accuracy: 0.0400 - val_loss: 4.3279 - val_accuracy: 0.0333
Epoch 2/10
10/10 [==============================] - 0s 29ms/step - loss: 4.3414 - accuracy: 0.0408 - val_loss: 4.2882 - val_accuracy: 0.0433
Epoch 3/10
10/10 [==============================] - 0s 30ms/step - loss: 4.3084 - accuracy: 0.0367 - val_loss: 4.2579 - val_accuracy: 0.0633
Epoch 4/10
10/10 [==============================] - 0s 30ms/step - loss: 4.2559 - accuracy: 0.0517 - val_loss: 4.2293 - val_accuracy: 0.0733
Epoch 5/10
10/10 [==============================] - 0s 28ms/step - loss: 4.2392 - accuracy: 0.0558 - val_loss: 4.1924 - val_accuracy: 0.0933
Epoch 6/10
10/10 [==============================] - 0s 29ms/step - loss: 4.1968 - accuracy: 0.0550 - val_loss: 4.1594 - val_accuracy: 0.1100
Epoch 7/10
10/10 [==============================] - 0s 30ms/step - loss: 4.1464 - accuracy: 0.0717 - val_loss: 4.0591 - val_accuracy: 0.1267
Epoch 8/10
10

In [43]:
model.save('/content/model1')

INFO:tensorflow:Assets written to: /content/model1/assets


In [46]:
import tensorflow as tf

In [48]:
new_model = tf.keras.models.load_model('/content/model1')

In [49]:
new_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1024)              25691136  
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               3

In [50]:
new_model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), callbacks=[mcp_save], batch_size=128)

Epoch 1/10
10/10 [==============================] - 1s 43ms/step - loss: 3.9076 - accuracy: 0.1133 - val_loss: 3.8739 - val_accuracy: 0.1600
Epoch 2/10
10/10 [==============================] - 0s 18ms/step - loss: 3.8103 - accuracy: 0.1167 - val_loss: 3.7619 - val_accuracy: 0.1400
Epoch 3/10
10/10 [==============================] - 0s 19ms/step - loss: 3.7715 - accuracy: 0.1108 - val_loss: 3.7063 - val_accuracy: 0.1567
Epoch 4/10
10/10 [==============================] - 0s 18ms/step - loss: 3.7056 - accuracy: 0.1242 - val_loss: 3.6569 - val_accuracy: 0.1700
Epoch 5/10
10/10 [==============================] - 0s 20ms/step - loss: 3.6602 - accuracy: 0.1250 - val_loss: 3.6284 - val_accuracy: 0.1800


KeyboardInterrupt: ignored

In [53]:
from google.colab import files

In [62]:
!zip -r /content/model_video_trained.zip /content/model1

  adding: content/model1/ (stored 0%)
  adding: content/model1/saved_model.pb (deflated 89%)
  adding: content/model1/variables/ (stored 0%)
  adding: content/model1/variables/variables.index (deflated 66%)
  adding: content/model1/variables/variables.data-00000-of-00001 (deflated 16%)
  adding: content/model1/assets/ (stored 0%)


In [64]:
files.download('/content/model_video_trained.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>